# Behandling 1

In [1]:
import requests 
import json
import time 
import tqdm
import pandas as pd

df = pd.read_csv('Boliga Scraped - Rådata.csv')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 500
df.sort_values(by = ['municipality'], axis = 0 );

In [55]:
#Oprydning
df.drop(df.loc[df['municipality']==0].index, inplace = True)
df.drop(df.loc[df['price']<=100].index, inplace = True)
df.drop(df.loc[df['latitude']==0].index, inplace = True)
df.drop(df.loc[df['longitude']==0].index, inplace = True)

#Unique municipality df
kommune = df['municipality'].unique()
df_kom = pd.DataFrame(kommune, columns = ['Kommunenr'])

In [61]:
import scraping_class
logfile = 'logfile_boliga.csv'
connector = scraping_class.Connector(logfile)

import time
def kommune_func(kommune_kode):
    url = lambda q: 'https://dawa.aws.dk/kommuner/{}'.format(q)
    resp, call_id = connector.get(url(kommune_kode), 'Kommunedata')
    time.sleep(0.5)
    
    if not resp.status_code == 200:
        return ''
    resp = resp.json()
    kommune = resp['navn']
    
    return kommune

In [69]:
import scraping_class
logfile = 'Boligscrape_log.csv'
connector = scraping_class.Connector(logfile)

import time
def region_func(kommune_kode):
    url = lambda q: 'https://dawa.aws.dk/kommuner/{}'.format(q)
    resp, call_id = connector.get(url(kommune_kode), 'Kommunedata')
    time.sleep(0.5)
    
    if not resp.status_code == 200:
        return ''
    resp = resp.json()
    region = resp['region']['navn']
    
    return region

In [63]:
df_kom['Kommune'] = df_kom.Kommunenr.apply(kommune_func)

In [70]:
df_kom['Region'] = df_kom.Kommunenr.apply(region_func)

In [72]:
df = df.merge(df_kom, how='left', left_on=['municipality'],
                           right_on=['Kommunenr'], validate = 'm:1')

df = df.drop(columns = ['Kommunenr'], axis = 1)

In [139]:
# Convert Prop_type (number) to descriptive labels
prop_type = {'Prop_Type': {'Villa': 1, 'Rækkehus': 2, 'Ejerlejlighed': 3,
            'Fritidshus': 4, 'Andelsbolig': 5, 'Landejendom': 6,
            'Helårsgrund': 7, 'Fritidsgrund': 8, 'Villalejlighed': 9}}

prop_type_df = pd.DataFrame(prop_type).reset_index()

df = df.merge(prop_type_df, how='left', left_on=['propertyType'],
                           right_on=['Prop_Type'], validate = 'm:1')

df = df.drop(columns = ['Prop_Type'], axis = 1)
df.rename(columns={'index':'Type'}, inplace=True)
#Drop missing in Type
df = df[df.Type.isnull() != True]

In [6]:
df.to_csv('Boliga - Behandling 1 - Kom, Reg, Type.csv', index = False)